In [1]:
import pandas as pd
import os
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
import requests
from bs4 import BeautifulSoup
from datetime import datetime, time,timedelta
import locale
from io import BytesIO
from email.mime.application import MIMEApplication
import calendar

# DECLARATION VARIABLE

In [ ]:
URL_OJK = 'https://www.ojk.go.id/id/Regulasi/Default.aspx'
URL_BI = 'https://www.bi.go.id/id/publikasi/peraturan/Default.aspx'
URL_LPS = 'https://lps.go.id/plps/'
